# Logistic Regression from Scratch

## Imports

In [1]:
import math
import random
import numpy as np
import pandas as pd
from collections import Counter
from typing import List, Tuple, TypeVar
import tqdm

Type definitions:

In [2]:
Vector = np.ndarray

**Model:**
$$
log(\frac{Prob(classification)}{1-Prob(classification)}) = y_i = \beta_0 + \beta_1 x_{i1} + \beta_2 x_{i2} + \dots + \beta_k x_{ik} + \epsilon
\\
probability(classification) = \frac{e^{y_i}}{1+e^{y_i}}
$$

## General Assumptions

- Features (independent variable) are linearly independent and it has a linear relationship with the dependent variable

- Features are all uncorrelated with epslon (error of model)

- For the sake of implementition, we assume the value of epsilon is zero on average (hence ignoring its effects)


**Implemented Model**

```python
y_i = beta_0 + beta_1 * x_i_1 + beta_2 * x_i_2 + ..........+ beta_k * x_i_k + epsilon

Beta = [b_0, b_1, b_2, ....... b_k]

X_i = [x_i_1, x_i_2, ........ x_i_k]

Y_i = Beta * X_i + beta_0 + epsilon   # dot product of Beta_vector and x_i
```

![alt text](https://datatab.net/assets/tutorial/Logistic-function.png)

## Implementaion

In [3]:
# LOGISTIC FUNCTION: Modified to fix floating point overflow

def logistic(x: float) -> float:
    """Applying logistic function on x"""
    if x < -709:
        return 0.0
    elif x > 709:
        return 1.0
    return 1.0 / (1 + np.exp(-x))

As defined in the **logistic function** (Sigmoid function) is a type of *logarithmic function*. As input gets large and positive, it gets closer and closer to 1.as input gets larger and negative,it gets closer to zero. And its derivative could be calculated by:

using this to fit the model:
`y_i = F( x_i * beta) + epslon`

F = Logistic function

recall we did in linear regression,using squared error function we calculated beta that minimizes the error or maximizes the likelyhood of data. Here we have to use gradient descent to maximixe the likelyhood of data.

intrepeting the model: y_i = F(x_i * beta)

given some beta, each y_i should equal to one with a probability F(x_i*beat) and equal to zero with probability 1-F(x_i * beta)

now, the PDF for y_i is:

$P(y_i|x_i, beta) = (F(x_i*beta)**y_i) • (1-F(x_i*beta))**(1-y_i)$  ---------------> eq(1)

since y_i = 0:
1 - F(x_i * beta)

since y_i = 1:
F(x_i * beta)


applying log on eq(1)

$logL(beta|x_i, y_i) = y_i • log(F(x_i*beta)) + (1 - y_i) • log(1 - F(x_i*beta))$

now log is a strictly increasing function, any beta maximizes the likelihood of data also maximizes the likelyhood of log likelyhood. since gradient descent minimizes things,we work with negative log likelyhood, because maximizing likelyhood is same as minimizing its negative.

"Log likelyhood function"

In [4]:
def _negative_log_likelihood(x: Vector, y: float, beta: Vector) -> float:
    """calculates the negative log likelihood for one data point"""
    if y == 1:
        return -np.log(logistic(x.dot(beta)))
    else:
        return -np.log(1 - logistic(x.dot(beta)))


# assuming different observations (data points) are independent from each other
# overoll log likelihood is the sum of individual log likelihoods.

In [5]:
def negative_log_likelihood(xs: List[Vector], ys: Vector, beta: Vector) -> float:
    return np.array([_negative_log_likelihood(x, y, beta) for x, y in zip(xs, ys)]).sum()

In [6]:
# now we have to find the gradient
def _negative_log_partial_j(x: Vector, y: float, beta: Vector, j: int) -> float:
    """calculates the j-th partial derivative at one data point,here j is the index of parameters in a single data point"""
    return -(y - logistic(x.dot(beta))) * x[j]

In [7]:
def _negative_log_gradient(x: Vector, y: float, beta: Vector) -> Vector:
    """gradient for one data point"""
    return np.array([_negative_log_partial_j(x, y, beta, j) for j in range(len(beta))])

In [8]:
def negative_log_gradient(xs: List[Vector], ys: Vector, beta: Vector) -> Vector:
    return np.array([_negative_log_gradient(x, y, beta) for x, y in zip(xs, ys)]).sum(axis=0)

## Rescaling

In ML,when dimensions arent comparable with each other,we may have to "rescale" our data so that each diamention has mean zero and standerd deviation 1. This effectivly get rid of units converting each dimension to standard deviations from mean. so to begin with we need to calculate mean and std deviation too

### pre-reqs

In [9]:
def squared_error(xs_error: Vector) -> Vector:
    """Returns the squared value of each element in the list of error"""
    return np.array([x_i**2 for x_i in xs_error])

In [10]:
def error_from_mean(xs: Vector) -> Vector:
    x_bar = xs.mean()
    return np.array([x - x_bar for x in xs])

In [11]:
def variance(xs: Vector) -> float:
    """accepts a list of data points and returns its variance from mean"""
    n = len(xs)
    xs_error = error_from_mean(xs)
    xs_error_squared = squared_error(xs_error)
    return sum(xs_error_squared) / (n - 1)

In [12]:
def std_deviation(xs: Vector) -> float:
    """accepts a list and returns its Standerd deviation from mean"""
    return math.sqrt(variance(xs))

### function def

In [13]:
def calculate_scale_parameters(data: List[Vector]) -> Tuple[Vector, Vector]:
    """returns the mean and std deviation of each position"""
    dim = len(data[0])
    means = np.mean(data, axis=0)
    stdevs = np.array([np.array([vector[i] for vector in data]).std() for i in range(dim)])
    return means, stdevs

In [14]:
def scale(data: List[Vector]) -> List[Vector]:
    """rescale the input data so that each position has a mean of 0 and std dev=1"""
    dim = len(data[0])
    means, stdevs = calculate_scale_parameters(data)
    rescaled = [v[:] for v in data]  # make a copy of each vector
    for v in rescaled:
        for i in range(dim):
            if stdevs[i] > 0:
                v[i] = (v[i] - means[i]) / stdevs[i]

    return rescaled

Gradient Descent

In [15]:
def descent_one_step(v: Vector, gradient: Vector, learning_rate: float) -> Vector:
    """Starts from v and moves a step units in oppsite direction of gradient"""
    assert len(v) == len(gradient), "The vector and its gradient are of different lengths"
 
    step = (-learning_rate) * gradient
    return v + step

## Train-Test split

In [16]:
# generic type to represnt a data point
X = TypeVar("X")
Y = TypeVar("Y")


def split_data(data: List[X], prob: float, seed:int = None) -> Tuple[List[X], List[X]]:
    """Split the given data into fractions of prob('prob' is fraction of training data set) given [[prob, 1-prob]]"""
    data = data.copy()
    random.seed(seed)
    random.shuffle(data)
    cut = int(len(data) * prob)
    return data[:cut], data[cut:]


def train_test_split(
    xs: List[X], ys: List[Y], test_pct: float, seed:int=None
) -> Tuple[List[X], List[X], List[Y], List[Y]]:
    """Split them by using indeices"""

    idxs = [i for i in range(len(xs))]
    train_idxs, test_idxs = split_data(idxs, 1 - test_pct, seed=seed)

    return (
        np.array([xs[i] for i in train_idxs]),      # x train
        np.array([xs[i] for i in test_idxs]),       # x test,
        np.array([ys[i] for i in train_idxs]),      # y train, 
        np.array([ys[i] for i in test_idxs]),       # y test
    )

## Fitting model into data: (*logistic regression*)

In [17]:
# # Fitting the Logistic Regression model

# random.seed(0)
# xs = features.values
# rescaled_xs = scale(xs)
# ys = target['Purchased'].values
# x_train, x_test, y_train, y_test = train_test_split(rescaled_xs, ys, 0.33)

# learning_rate = 0.001


# # Pick a random starting point
# beta = np.array([random.random() for _ in range(len(xs[0]))])

# with tqdm.trange(5000) as t:
#     for epoch in t:
#         gradient = negative_log_gradient(x_train, y_train, beta)
#         beta = descent_one_step(beta, gradient, learning_rate)
#         loss = negative_log_likelihood(x_train, y_train, beta)
#         t.set_description(f"loss = {loss:.5f}")

# print(beta)

In [18]:
def fit(x_train: List[X], y_train:List[Y], learning_rate:float = 0.001, max_iter:int = 5000):
    """Returns the Beta for with the log-likelihood is maximum. Using gradient descent to minimize the negative of maximum log likelihood function"""
    
    # Pick a random starting point
    beta = np.array([random.random() for _ in range(len(x_train[0]))])

    with tqdm.trange(max_iter) as t:
        for epoch in t:
            gradient = negative_log_gradient(x_train, y_train, beta)
            beta = descent_one_step(beta, gradient, learning_rate)
            loss = negative_log_likelihood(x_train, y_train, beta)
            t.set_description(f"loss = {loss:.5f}")

    return beta

# Testing with a dataset

In [19]:
# ! kaggle datasets download -d dragonheir/logistic-regression
# ! unzip -o "logistic-regression.zip"
# ! rm "logistic-regression.zip"

In [ ]:
df = pd.read_csv("data/Social_Network_Ads.csv")

In [21]:
df['is_male'] = df['Gender'].map({'Male': 1, 'Female': 0})
df

,User ID,Gender,Age,EstimatedSalary,Purchased,is_male
0,15624510,Male,19,19000,0,1
1,15810944,Male,35,20000,0,1
2,15668575,Female,26,43000,0,0
3,15603246,Female,27,57000,0,0
4,15804002,Male,19,76000,0,1
...,...,...,...,...,...,...
395,15691863,Female,46,41000,1,0
396,15706071,Male,51,23000,1,1
397,15654296,Female,50,20000,1,0
398,15755018,Male,36,33000,0,1


In [22]:
features = df[["Age", "EstimatedSalary", "is_male"]]
target = df[["Purchased"]]

In [23]:
features

,Age,EstimatedSalary,is_male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [24]:
target

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


## Training

In [25]:
# Fitting the Logistic Regression model

xs = features.values
rescaled_xs = scale(xs)
ys = target['Purchased'].values

x_train, x_test, y_train, y_test = train_test_split(rescaled_xs, ys, 0.33, seed=0)

beta = fit(x_train=x_train, y_train=y_train, learning_rate = 0.001, max_iter = 5000)
beta

loss = 144.84446:   0%|          | 13/5000 [00:00<00:38, 129.42it/s]

loss = 134.29922: 100%|██████████| 5000/5000 [00:34<00:00, 144.62it/s]


array([ 2.13206465,  0.66228359, -0.91532749])

## Testing

### Testing on training data

In [26]:
beta_vector = np.array(beta)
beta_vector

array([ 2.13206465,  0.66228359, -0.91532749])

In [27]:
X = np.array(x_train)
X

array([[-1, -1,  0],
       [ 0, -1,  1],
       [-1,  0,  0],
       [-1,  0,  1],
       [-1,  0,  1],
       [-1,  0,  0],
       [ 0, -1,  0],
       [ 0,  1,  1],
       [ 0,  0,  1],
       [ 1,  0,  0],
       [ 0,  0,  1],
       [-1, -1,  0],
       [-1,  0,  0],
       [ 0,  0,  1],
       [ 0, -1,  1],
       [ 0,  0,  1],
       [ 0,  0,  1],
       [-1, -1,  1],
       [ 0,  0,  0],
       [ 0,  0,  1],
       [ 0,  0,  1],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 1,  1,  0],
       [ 0,  0,  1],
       [-1,  0,  0],
       [ 0,  0,  0],
       [ 0, -1,  0],
       [ 2,  0,  1],
       [ 0,  2,  0],
       [ 1,  1,  0],
       [ 0,  0,  0],
       [ 0,  0,  1],
       [-1,  0,  0],
       [ 0,  0,  0],
       [ 1,  1,  0],
       [ 0,  0,  0],
       [ 0,  0,  1],
       [ 0,  0,  0],
       [ 0,  0,  1],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  1],
       [ 1,  0,  1],
       [-1, -1,  1],
       [ 0,  0,  1],
       [ 0,  

In [28]:
log_scale_Y = X @ beta_vector
log_scale_Y

array([-2.79434824, -1.57761108, -2.13206465, -3.04739214, -3.04739214,
       -2.13206465, -0.66228359, -0.2530439 , -0.91532749,  2.13206465,
       -0.91532749, -2.79434824, -2.13206465, -0.91532749, -1.57761108,
       -0.91532749, -0.91532749, -3.70967573,  0.        , -0.91532749,
       -0.91532749,  0.        ,  0.        ,  0.        ,  2.79434824,
       -0.91532749, -2.13206465,  0.        , -0.66228359,  3.34880181,
        1.32456719,  2.79434824,  0.        , -0.91532749, -2.13206465,
        0.        ,  2.79434824,  0.        , -0.91532749,  0.        ,
       -0.91532749,  0.        ,  0.        , -0.91532749,  1.21673716,
       -3.70967573, -0.91532749,  0.66228359, -0.91532749, -2.13206465,
       -0.91532749,  0.        ,  0.66228359,  0.        ,  1.46978106,
       -0.91532749,  1.32456719, -0.91532749, -0.2530439 ,  0.        ,
       -0.91532749, -0.91532749,  2.13206465, -0.91532749,  0.66228359,
       -3.70967573,  2.79434824, -0.91532749, -0.91532749,  2.79

In [29]:
# Applying Logistic function on to the log of odds scale to get the probabilty (prediction)
Y = np.vectorize(logistic)(log_scale_Y)
Y

array([0.05763035, 0.17113408, 0.10601915, 0.0453302 , 0.0453302 ,
       0.10601915, 0.34022682, 0.43707443, 0.28591091, 0.89398085,
       0.28591091, 0.05763035, 0.10601915, 0.28591091, 0.17113408,
       0.28591091, 0.28591091, 0.02390025, 0.5       , 0.28591091,
       0.28591091, 0.5       , 0.5       , 0.5       , 0.94236965,
       0.28591091, 0.10601915, 0.5       , 0.34022682, 0.96606558,
       0.78994056, 0.94236965, 0.5       , 0.28591091, 0.10601915,
       0.5       , 0.94236965, 0.5       , 0.28591091, 0.5       ,
       0.28591091, 0.5       , 0.5       , 0.28591091, 0.77148884,
       0.02390025, 0.28591091, 0.65977318, 0.28591091, 0.10601915,
       0.28591091, 0.5       , 0.65977318, 0.5       , 0.81302411,
       0.28591091, 0.78994056, 0.28591091, 0.43707443, 0.5       ,
       0.28591091, 0.28591091, 0.89398085, 0.28591091, 0.65977318,
       0.02390025, 0.94236965, 0.28591091, 0.28591091, 0.94236965,
       0.28591091, 0.63516823, 0.5       , 0.81302411, 0.43707

Categorizing(labelling) results with probs >= 0.5 as '1' *(Using threshold as 0.5)*

In [30]:
predicted_purchased = Y >= 0.5

In [31]:
correctly_predicted = predicted_purchased == y_train
correctly_predicted

array([ True, False,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True,  True, False,
       False, False,  True, False, False,  True,  True,  True,  True,
        True,  True, False, False, False,  True,  True,  True,  True,
        True,  True,  True, False, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
       False,  True, False, False,  True,  True,  True, False, False,
        True, False,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True, False, False, False,
        True,  True, False,  True, False, False,  True, False,  True,
       False, False,  True,  True,  True, False,  True, False,  True,
        True, False,  True,  True, False,  True,  True, False,  True,
        True,  True,

In [32]:
accuracy = correctly_predicted.sum() / len(correctly_predicted)
accuracy

np.float64(0.6753731343283582)

## Goodness of fit

Suppose we predict paid account when ever proabability is greater than 0.5

![Confusion Matrix](https://miro.medium.com/v2/resize:fit:1400/1*pOtBHai4jFd-ujaNXPilRg.png)

In [33]:
# Assuming threshold to be probability >= 0.5 = True
true_positives = false_positives = true_negatives = false_negatives = 0

for x_i, y_i in zip(x_train, y_train):
    prediction = logistic(np.dot(beta, x_i))
    if y_i == 1 and prediction >= 0.5:
        true_positives += 1  # paid and we predicted paid-TP
    elif y_i == 1:
        false_negatives += 1  # paid and predicted unpaid -FN
    elif prediction >= 0.5:
        false_positives += 1  # unpaid but predicted paid-FP
    else:
        true_negatives += 1  # unpaid and predicted unpaid-TN


accuracy = true_positives + true_negatives / (
    true_positives + false_positives + true_negatives + false_negatives
)
precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)

accuracy, precision, recall

(61.44776119402985, 0.5126050420168067, 0.6777777777777778)